In [34]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
audience = pd.read_csv('audiences_info.csv')
audience.fillna(value=False, inplace=True)

Name                 0
id_Number            0
row_Number           0
seat_Number          0
auditorium_Number    0
test_result          0
score                0
dtype: int64

## functions for Q1

In [37]:
result = {i:{'initial':None, 'infect_time':None, 'positive': None, 'score':None, 'name':None} for i in range(1,7)}

In [38]:
def get_aud_info(aud_num):
    info = audience[audience['auditorium_Number']==aud_num]
    row = max(info['row_Number'])
    seat = max(info['seat_Number'])
    aud = np.ones((row,seat))*(-1)
    score = np.ones((row,seat))*(-1)
    infect = np.ones((row,seat))*(-1)
    name = [[None for j in range(seat)] for i in range(row)]
    positive = []
    
    infected = info.loc[info['test_result']==True,['row_Number','seat_Number']]
    healthy = info.loc[info['test_result']==False,['row_Number','seat_Number']]
    
    for r,c in infected.values:
        aud[r-1,c-1] = 1
        infect[r-1,c-1] = 0
    for r,c in healthy.values:
        aud[r-1,c-1] = 0
    for n, ID, r, c, s in info.loc[:,['Name','id_Number','row_Number','seat_Number','score']].values:
        score[r-1,c-1] = math.floor(s * 0.06)
        name[r-1][c-1] = (n,ID)
    
    result[aud_num]['initial'] = aud
    result[aud_num]['infect_time'] = infect
    result[aud_num]['positive'] = infected.values-1
    result[aud_num]['score'] = score
    result[aud_num]['name'] = name

In [39]:
def get_infect_time(aud_num):
    
    aud = result[aud_num]['initial']
    infect = result[aud_num]['infect_time']
    score = result[aud_num]['score']
    row, seat = aud.shape
    
    for start in result[aud_num]['positive']:
        queue = [start]
        while len(queue) != 0:
            r,c = queue[0]
            queue = queue[1:]
            for n_r, n_c in [(r+1,c),(r-1,c),(r,c+1),(r,c-1)]:
                if (0 <= n_r < row) and ( 0 <= n_c < seat):
                    if aud[n_r, n_c]>=0: # if there is people there
                        # not infected yet or could have been infected earlier
                        if (infect[n_r, n_c]<0) or (infect[n_r, n_c] > infect[r,c] + score[n_r, n_c]):
                            infect[n_r, n_c] = infect[r,c] + score[n_r, n_c]
                            # which means other people could have been infected earlier as well
                            queue.append((n_r, n_c))

In [40]:
get_aud_info(1)
get_infect_time(1)

1 (50, 68)
number of infected people: 186
number of healthy people: 2376


42.0

In [41]:
get_aud_info(2)
get_infect_time(2)

2 (59, 66)
number of infected people: 245
number of healthy people: 2745


43.0

In [42]:
get_aud_info(3)
get_infect_time(3)

3 (39, 47)
number of infected people: 118
number of healthy people: 1276


34.0

In [43]:
get_aud_info(4)
get_infect_time(4)

4 (68, 68)
number of infected people: 286
number of healthy people: 3206


38.0

In [44]:
get_aud_info(5)
get_infect_time(5)

5 (46, 62)
number of infected people: 221
number of healthy people: 1933


39.0

In [45]:
get_aud_info(6)
get_infect_time(6)

6 (43, 43)
number of infected people: 113
number of healthy people: 1301


49.0

# Question 2

In [46]:
df = pd.DataFrame(columns=('Name', 'id_Number', 'ExposureTime'))
idx=0
for a in range(1,7):
    aud = result[a]['initial']
    name = result[a]['name']
    infect = result[a]['infect_time']
    
    for i in range(len(aud)):
        for j in range(len(aud[0])):
            if infect[i,j ] > 0:
                df.loc[idx] = [name[i][j][0], name[i][j][1], infect[i,j]]
                idx += 1


df.sort_values('ExposureTime',ascending=False,inplace=True)
df.head(n=15)

,Name,id_Number,ExposureTime
12228,Steven Henley,91902201809274958,49.0
12256,Erin Serrell,10707200203039606,45.0
4459,David Ferber,79461195304211842,43.0
1973,Paul Abbott,32979197407141829,42.0
13,Ronald Lamb,11939198412054942,42.0
12255,Jacqueline Key,11606201502206319,41.0
2381,James Archuleta,78123198703168356,41.0
280,Nancy Avella,34124193211171765,41.0
2376,Stacy Wang,22460201307164487,41.0
12286,Brandon Allison,79418195607227075,41.0


In [47]:
df.to_csv('alert_info.csv',index=None)